In [1]:
import os, sys
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import json
import ete3
import subprocess
import re
import multiprocessing as mp
import pysam
from tqdm import tqdm
from Bio import SeqIO

from vcfClass import VCF
from figure_constants import *

loading subjects...
loading samples...
loading segments...
loading genes...
loading SNPs...
loading transmission pairs...
loading transmission segments...
loading transmission SNPs...


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (58) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/mnt/d/orchards/figure_constants.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HA_add_on['antigenic_product'] = 'HA'


1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [2]:
#Load VCF files and annotate. Assumes already filtered.

print ('loading VCFs..')
VCFs = [VCF(x) for x in vcffiles]

#Add annotation info
print ('Annotating...')
for allSNPsVCF, g, r in zip(allSNPsVCFs, gtfFiles, consensusReferences):
    print (r)
    regions = allSNPsVCF.annotate(g,r)

loading VCFs..
Annotating...
/mnt/d/orchards/h1n1/orchards_run19H3N2/A_Singapore_INFIMH-16-0019_2016/consensus/A_Singapore_INFIMH-16-0019_2016_consensus_noambig.fasta
/mnt/d/orchards/h1n1/orchards_run19H3N2/A_Hong_Kong_4801_2014_EPI834581/consensus/A_Hong_Kong_4801_2014_EPI834581_consensus_noambig.fasta
/mnt/d/orchards/h1n1/orchards_run19/A_Michigan_45_2015_H1N1_18/consensus/A_Michigan_45_2015_H1N1_18_consensus_noambig.fasta
/mnt/d/orchards/h1n1/orchards_run19/A_Michigan_45_2015_H1N1_19/consensus/A_Michigan_45_2015_H1N1_19_consensus_noambig.fasta
/mnt/d/orchards/h1n1/orchards_runB/B_Phuket_3073_2013_17/consensus/B_Phuket_3073_2013_17_consensus_noambig.fasta
/mnt/d/orchards/h1n1/orchards_runB/B_Phuket_3073_2013_18/consensus/B_Phuket_3073_2013_18_consensus_noambig.fasta


In [3]:
def one_hot_encode(seq):
    mapping = dict(zip("ACGT", range(4)))    
    seq2 = [mapping[i] for i in seq]
    return np.eye(4)[seq2]

def create_ref_array(vcf):
    concatrefseq = ""
    segStarts = dict()
    segCoords = dict()
    runningtally = 0
    for chrom, seq in vcf.reference.items():
        segStarts[chrom.split('_')[-1]] = runningtally
        segCoords[chrom.split('_')[-1]] = (runningtally, runningtally+len(seq))
        runningtally += len(seq)
        concatrefseq += seq
    ref_array = one_hot_encode(concatrefseq)
    ref_array = np.repeat(ref_array[np.newaxis,:,:], len(vcf.samples), axis=0)
    return ref_array

In [4]:
distances = list()
for vcf in VCFs:
    n, samples, SNP_locs = vcf.to_numpy(cutoff_freq = .01)
    ref_array = create_ref_array(vcf)
    values_not_in_empty_nuc = np.repeat((np.sum(n, axis=2) != 0)[:,:,np.newaxis], 4, axis=2)
    frequencies = np.where(values_not_in_empty_nuc, n/np.sum(n,axis=2,keepdims=True), n)
    freq_array = np.where(values_not_in_empty_nuc, frequencies, ref_array)
    distances.extend([(sampleA, sampleB, np.sum(np.abs(freq_array[i]-freq_array[j]))/2) 
                 for i, sampleA in enumerate(samples) 
                 for j, sampleB in enumerate(samples) 
                 if sampleA != sampleB])

<ipython-input-4-6464aeab065f>:6: RuntimeWarning: invalid value encountered in true_divide
  frequencies = np.where(values_not_in_empty_nuc, n/np.sum(n,axis=2,keepdims=True), n)
<ipython-input-4-6464aeab065f>:6: RuntimeWarning: invalid value encountered in true_divide
  frequencies = np.where(values_not_in_empty_nuc, n/np.sum(n,axis=2,keepdims=True), n)
<ipython-input-4-6464aeab065f>:6: RuntimeWarning: invalid value encountered in true_divide
  frequencies = np.where(values_not_in_empty_nuc, n/np.sum(n,axis=2,keepdims=True), n)
<ipython-input-4-6464aeab065f>:6: RuntimeWarning: invalid value encountered in true_divide
  frequencies = np.where(values_not_in_empty_nuc, n/np.sum(n,axis=2,keepdims=True), n)
<ipython-input-4-6464aeab065f>:6: RuntimeWarning: invalid value encountered in true_divide
  frequencies = np.where(values_not_in_empty_nuc, n/np.sum(n,axis=2,keepdims=True), n)
<ipython-input-4-6464aeab065f>:6: RuntimeWarning: invalid value encountered in true_divide
  frequencies = np.

In [5]:
distances = pd.DataFrame(distances, columns=['index','contact','distance'])

In [6]:
unique_distances = distances.drop_duplicates(subset='distance')

In [7]:
distances.to_csv('/mnt/d/orchards/H1N1/figures/allvsall.tsv', sep='\t')